In [2]:
import duckdb
import shapely
import shapely.geometry

In [3]:
conn = duckdb.connect(':memory:')

In [4]:
conn.execute("INSTALL spatial; LOAD spatial;")

In [5]:
conn.execute("SELECT ST_AsText(ST_Point(1, 2));").fetchall()

[('POINT (1 2)',)]

In [6]:
conn.execute("CREATE TABLE points (point GEOMETRY);")

In [13]:
hex = shapely.geometry.Point(1, 2).wkb_hex
hex

'0101000000000000000000F03F0000000000000040'

In [80]:
hex = shapely.geometry.MultiPoint(points=[(1, 2), (3, 4), (10, 20)]).wkb
hex

b'\x01\x04\x00\x00\x00\x03\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00$@\x00\x00\x00\x00\x00\x004@'

In [29]:
conn.execute("INSERT INTO points VALUES (ST_GeomFromWKB(?));", [hex])
conn.execute("SELECT ST_AsText(point) FROM points;").fetchall()

[('POINT (1 2)',),
 ('POINT (1 2)',),
 ('POINT (1 2)',),
 ('MULTIPOINT Z (1 2 5, 3 4 6)',),
 ('MULTIPOINT Z (1 2 5, 3 4 6)',),
 ('MULTIPOINT Z (1 2 5, 3 4 6)',),
 ('MULTIPOINT Z (1 2 5, 3 4 6)',)]

In [18]:
conn.execute("INSERT INTO points VALUES (?);", [shapely.geometry.Point(1, 2).wkt])

In [81]:
conn.execute("select st_astext(ST_ConvexHull(st_geomfromwkb(?)));", [shapely.geometry.MultiPoint(points=[(1, 2), (3, 4), (10, 20)]).wkb]).fetchall()

[('POLYGON ((1 2, 10 20, 3 4, 1 2))',)]

In [13]:
conn.execute("SELECT ST_AsText(point) FROM points;").fetchall()

[('POINT (1 2)',)]

In [70]:
conn.sql("SELECT unnest({'a': [34, 45], 'b': [4,5]});")

┌──────────┬─────────┐
│    a     │    b    │
│ int32[]  │ int32[] │
├──────────┼─────────┤
│ [34, 45] │ [4, 5]  │
└──────────┴─────────┘

In [74]:
conn.execute(
    "SELECT ST_astext(points), index "
    "FROM (SELECT ST_GeomFromWKB(UNNEST(?)), UNNEST(?)) AS ViewArea(points, index) "
, [[hex, hex], [2, 3]]).fetchall()

[('MULTIPOINT Z (1 2 5, 3 4 6)', 2), ('MULTIPOINT Z (1 2 5, 3 4 6)', 3)]

In [19]:
('?,' * 19)[:-1]

'?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?'

In [ ]:
try:
    conn.execute("insert into points values ('test')")
except duckdb.Error as e:
    print(e)

Conversion Error: Expected geometry type at position '0' near: 't'|<---


In [24]:
import pandas
df = pandas.DataFrame({'point': [1]})
conn.from_df(df)

┌───────┐
│ point │
│ int64 │
├───────┤
│     1 │
└───────┘

In [86]:
import pandas as pd
conn.register('tmp', pd.DataFrame(data={
    'test': [1, 2, 3, 4],
    'test2': [5, 6, 7, 8],
}))
conn.sql('select * from tmp')

┌───────┬───────┐
│ test  │ test2 │
│ int64 │ int64 │
├───────┼───────┤
│     1 │     5 │
│     2 │     6 │
│     3 │     7 │
│     4 │     8 │
└───────┴───────┘